In [1]:
import preprocessing
import pandas as pd
import numpy as np
import importlib
from g2p_en import G2p # https://github.com/Kyubyong/g2p

In [2]:
celex_dict_file = "Data/epw.cd"
filename = "Data/2016_all_words_no_audio.pickle"
hom_filename = "Data/hom.csv"

In [3]:
df = preprocessing.read_dataframe(filename, remove_pauses=True, remove_errors=True, preprocessing=True, drop_error_columns=False)

read dataframe from Data/2016_all_words_no_audio.pickle
Preprocessing: extract pause information...
Remove pauses from data!
Preprocessing: apply word preprocessing...
Preprocessing: calculate word duration...
Preprocessing: calculate word frequency...
Preprocessing: extract context information...
Preprocessing: calculate letter length...
Preprocessing: calculate contextual predictability...
(18864660, 25) RangeIndex(start=0, stop=18864660, step=1)


In [200]:
df

KeyboardInterrupt: 

In [12]:
source_files = ["2016-12-17_1330_US_KCET_Asia_Insight", "2016-10-25_2300_US_KABC_Eyewitness_News_4PM"]

In [13]:
sub_df = pd.read_csv("sub_df.csv", index_col="Unnamed: 0")
sub_df

,source_file,word,start,end,duration,label_type,mp4_error,aac_error,aac2wav_error,eafgz_error,...,prev_word_frequency,next_word,next_word_frequency,letter_length,prev_word_string,next_word_string,prev_word_string_frequency,next_word_string_frequency,cond_pred_prev,cond_pred_next
14828820,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,police,0.29,0.67,0.38,high-confidence,no-error,no-error,no-error,no-error,...,NaN,don't,32647.0,6,NaN,police-don't,NaN,37.0,NaN,0.001133
14828821,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,don't,0.67,0.88,0.21,high-confidence,no-error,no-error,no-error,no-error,...,18598.0,believe,9847.0,5,police-don't,don't-believe,37.0,540.0,0.001989,0.054839
14828822,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,believe,0.88,1.21,0.33,high-confidence,no-error,no-error,no-error,no-error,...,32647.0,the,932396.0,7,don't-believe,believe-the,540.0,793.0,0.016541,0.000850
14828823,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,the,1.22,1.40,0.18,high-confidence,no-error,no-error,no-error,no-error,...,9847.0,mother,3407.0,3,believe-the,the-mother,793.0,490.0,0.080532,0.143822
14828824,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,mother,1.40,1.79,0.39,high-confidence,no-error,no-error,no-error,no-error,...,932396.0,or,57737.0,6,the-mother,mother-or,490.0,21.0,0.000526,0.000364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17805800,2016-12-17_1330_US_KCET_Asia_Insight,2017,1657.55,1657.71,0.16,high-confidence,no-error,no-error,no-error,no-error,...,335519.0,as,89095.0,4,in-2017,2017-as,151.0,15.0,0.000450,0.000168
17805801,2016-12-17_1330_US_KCET_Asia_Insight,as,1659.25,1659.49,0.24,high-confidence,no-error,no-error,no-error,no-error,...,597.0,the,932396.0,2,2017-as,as-the,15.0,6909.0,0.025126,0.007410
17805802,2016-12-17_1330_US_KCET_Asia_Insight,the,1659.50,1659.65,0.15,high-confidence,no-error,no-error,no-error,no-error,...,89095.0,world's,1593.0,3,as-the,the-world's,6909.0,1265.0,0.077546,0.794099
17805803,2016-12-17_1330_US_KCET_Asia_Insight,world's,1659.65,1660.25,0.60,high-confidence,no-error,no-error,no-error,no-error,...,932396.0,largest,1971.0,7,the-world's,world's-largest,1265.0,218.0,0.001357,0.110604


In [4]:
homophones_in_data, gahls_homophones, gahls_homophones_missing_in_data = preprocessing.read_and_extract_homophones(hom_filename, df)

read Gahls Homophone data from Data/hom.csv
406 out of 412 homophones found in Data:
Homophone Pairs found in Data: 200
Homophones without Pair:  ['flowers', 'holes', 'moose', 'naval', 'pairs', 'taught']
Missing homophones: ['flours' 'mousse' 'navel' 'pears' 'taut' 'wholes']


In [21]:
homophones_in_data

,source_file,word,start,end,duration,label_type,mp4_error,aac_error,aac2wav_error,eafgz_error,...,next_word_string,prev_word_string_frequency,next_word_string_frequency,cond_pred_prev,cond_pred_next,has_pair,pron,celexPhon,pron_frequency,is_max
0,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,night,14.540000,14.940000,0.40,high-confidence,no-error,no-error,no-error,no-error,...,night-when,490.0,244.0,0.005139,0.005493,False,n2t,n2t,11,1
1,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,night,297.990000,298.300000,0.31,high-confidence,no-error,no-error,no-error,no-error,...,night-and,4410.0,754.0,0.194462,0.001763,False,n2t,n2t,11,1
2,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,night,737.299999,737.609999,0.31,high-confidence,no-error,no-error,no-error,no-error,...,night-he,4410.0,268.0,0.194462,0.002229,False,n2t,n2t,11,1
3,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,night,1187.000000,1187.520000,0.52,high-confidence,no-error,no-error,no-error,no-error,...,night-making,347.0,6.0,0.129914,0.000725,False,n2t,n2t,11,1
4,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,night,1195.000000,1195.350000,0.35,low-confidence,no-error,no-error,no-error,no-error,...,night-early,347.0,6.0,0.129914,0.001257,False,n2t,n2t,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,2016-12-17_1330_US_KCET_Asia_Insight,passed,1260.480000,1260.780000,0.30,high-confidence,no-error,no-error,no-error,no-error,...,passed-a,81.0,107.0,0.000674,0.000250,True,pBst,p#st,6,1
201,2016-12-17_1330_US_KCET_Asia_Insight,passed,1264.710000,1264.750000,0.04,low-confidence,no-error,no-error,no-error,no-error,...,passed-and,81.0,36.0,0.000674,0.000084,True,pBst,p#st,6,1
202,2016-12-17_1330_US_KCET_Asia_Insight,higher,1307.420000,1307.800000,0.38,high-confidence,no-error,no-error,no-error,no-error,...,higher-than,32.0,291.0,0.004683,0.010053,False,h2_TR,h2@R,1,1
203,2016-12-17_1330_US_KCET_Asia_Insight,waste,1384.379999,1384.669999,0.29,high-confidence,no-error,no-error,no-error,no-error,...,waste-one,41.0,3.0,0.000076,0.000051,False,w1st,w1st,1,1


In [5]:
def get_ARPAbet_phonetic_transcription(word_list):
    g2p = G2p()
    arpabet_word_list = []
    for word in word_list:
        transcription = g2p(word)
        arpabet_word_list.append(transcription)

    return arpabet_word_list


def get_english_phonology_from_celex(filename):
    phonology_dict = {"word":[], "disc":[], "clx":[]}
    with open(filename) as f:
        for line in f:
            line = line.strip().split("\\")
            word = line[1] # the word
            phonology_dict["word"].append(word)
            disc = line[6] # pronunciation in DISC notation, hyphens to mark syllable boundaries, inverted comma for primary stress and double quote for secondary stress (PhonStrsDISC)
            phonology_dict["disc"].append(disc)
            clx = line[8] # pronunciation in CELEX notation, with brackets (PhonSylBCLX)
            phonology_dict["clx"].append(clx)

    celex_phonology_dict = pd.DataFrame.from_dict(phonology_dict).drop_duplicates()
    celex_phonology_dict["disc_no_bound"] = celex_phonology_dict["disc"].apply(
        lambda x: x.replace("'", "").replace("-", ""))
    celex_phonology_dict["clx_no_bound"] = celex_phonology_dict["clx"].apply(
        lambda x: x.replace("[", "").replace("]", ""))
    return celex_phonology_dict



def get_celex_transcription(df, celex_phonology_dict):

    return df.merge(celex_phonology_dict[["word", "disc", "clx", "disc_no_bound", "clx_no_bound"]], how = "left", left_on=["word", "celexPhon"], right_on=["word","disc_no_bound"])


In [6]:
celex_phonology_dict = get_english_phonology_from_celex(celex_dict_file)

In [17]:
celex_phonology_dict

,word,disc,clx,disc_no_bound,clx_no_bound
0,a,'1,[eI],1,eI
2,A,'1,[eI],1,eI
4,AA,"""1-'1",[eI][eI],"""11",eIeI
6,AAs,"""1-'1z",[eI][eIz],"""11z",eIeIz
7,abaci,'{-b@-s2,[&][b@][saI],{b@s2,&b@saI
...,...,...,...,...,...
100620,Zouave,zu-'#v,[zu:][A:v],zu#v,zu:A:v
100621,Zouaves,zu-'#vz,[zu:][A:vz],zu#vz,zu:A:vz
100622,z's,'zEdz,[zEdz],zEdz,zEdz
100623,zucchini,zU-'ki-nI,[zU][ki:][nI],zUkinI,zUki:nI


In [7]:
homophones_in_data_celex_mapped = get_celex_transcription(homophones_in_data,celex_phonology_dict)

In [23]:
homophones_in_data_celex_mapped

,source_file,word,start,end,duration,label_type,mp4_error,aac_error,aac2wav_error,eafgz_error,...,cond_pred_next,has_pair,pron,celexPhon,pron_frequency,is_max,disc,clx,disc_no_bound,clx_no_bound
0,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,night,14.540000,14.940000,0.40,high-confidence,no-error,no-error,no-error,no-error,...,0.005493,False,n2t,n2t,11,1,'n2t,[naIt],n2t,naIt
1,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,night,297.990000,298.300000,0.31,high-confidence,no-error,no-error,no-error,no-error,...,0.001763,False,n2t,n2t,11,1,'n2t,[naIt],n2t,naIt
2,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,night,737.299999,737.609999,0.31,high-confidence,no-error,no-error,no-error,no-error,...,0.002229,False,n2t,n2t,11,1,'n2t,[naIt],n2t,naIt
3,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,night,1187.000000,1187.520000,0.52,high-confidence,no-error,no-error,no-error,no-error,...,0.000725,False,n2t,n2t,11,1,'n2t,[naIt],n2t,naIt
4,2016-10-25_2300_US_KABC_Eyewitness_News_4PM,night,1195.000000,1195.350000,0.35,low-confidence,no-error,no-error,no-error,no-error,...,0.001257,False,n2t,n2t,11,1,'n2t,[naIt],n2t,naIt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,2016-12-17_1330_US_KCET_Asia_Insight,passed,1260.480000,1260.780000,0.30,high-confidence,no-error,no-error,no-error,no-error,...,0.000250,True,pBst,p#st,6,1,'p#st,[pA:st],p#st,pA:st
201,2016-12-17_1330_US_KCET_Asia_Insight,passed,1264.710000,1264.750000,0.04,low-confidence,no-error,no-error,no-error,no-error,...,0.000084,True,pBst,p#st,6,1,'p#st,[pA:st],p#st,pA:st
202,2016-12-17_1330_US_KCET_Asia_Insight,higher,1307.420000,1307.800000,0.38,high-confidence,no-error,no-error,no-error,no-error,...,0.010053,False,h2_TR,h2@R,1,1,'h2-@R,[haI][@r*],h2@R,haI@r*
203,2016-12-17_1330_US_KCET_Asia_Insight,waste,1384.379999,1384.669999,0.29,high-confidence,no-error,no-error,no-error,no-error,...,0.000051,False,w1st,w1st,1,1,'w1st,[weIst],w1st,weIst


In [8]:
berndt_character_code = pd.read_csv("Data/celex_phonetic_character_code_berndt1987.csv", delimiter=";")
berndt_conditional_probs = pd.read_csv("Data/Conditional_Probabilities_for_Grapheme-to-Phoneme_Correspondences_Berndt1987.csv",delimiter=";")

In [27]:
berndt_character_code

,keyboard_compatible_phonetic_symbol,CELEX,g2p(ARPAbet),DISC,Example
0,ay,eI,EY1,1,ale
1,ae,&,AE1,{,add
2,ee,i:,IY1,i,bee
3,eh,E,EH1,E,end
4,er,@r*,ER0,@R,father
5,ai,aI,AY1,2,high
6,ih,I,IH1,I,bin
7,o,@U,OW1,5,boat
8,ah,O,AA1,Q,cot
9,aw,O,AA1,Q,soft


In [91]:
homophones_in_data_celex_mapped.columns

Index(['source_file', 'word', 'start', 'end', 'duration', 'label_type',
       'mp4_error', 'aac_error', 'aac2wav_error', 'eafgz_error', 'seg_error',
       'preceding_pause', 'subsequent_pause', 'word_frequency', 'prev_word',
       'prev_word_frequency', 'next_word', 'next_word_frequency',
       'letter_length', 'prev_word_string', 'next_word_string',
       'prev_word_string_frequency', 'next_word_string_frequency',
       'cond_pred_prev', 'cond_pred_next', 'has_pair', 'pron', 'celexPhon',
       'pron_frequency', 'is_max', 'disc', 'clx', 'disc_no_bound',
       'clx_no_bound'],
      dtype='object')

In [114]:
celex_phonology_dict[celex_phonology_dict.word == "gym"]

,word,disc,clx,disc_no_bound,clx_no_bound
39337,gym,'_Im,[dZIm],_Im,dZIm


In [9]:
arpabet_encoded_words = get_ARPAbet_phonetic_transcription(homophones_in_data_celex_mapped.word)
arpabet_used_in_data = set(sum(arpabet_encoded_words,[]))

#disc_encoded_words = list(homophones_in_data_celex_mapped.disc[pd.notnull(homophones_in_data_celex_mapped.disc)].str.replace("'","").str.split("-"))
#disc_used_in_data = set(sum(disc_encoded_words,[]))

#clx_encoded_words = list(homophones_in_data_celex_mapped.clx[pd.notnull(homophones_in_data_celex_mapped.clx)].str.replace("[","").str.split("]"))
#clx_used_in_data = set(filter(lambda x: x != "",sum(clx_encoded_words,[])))

In [10]:
arpabet_used_in_data - set(berndt_character_code["g2p(ARPAbet)"])

{'AA2', 'AO1', 'EH0', 'ER1', 'EY2', 'IH0', 'IY0', 'OW0', 'OW2'}

In [139]:
disc_characters_used_in_data = set(''.join(list(disc_used_in_data)))
disc_characters_for_berndts_encoding = set(''.join([str(i) for i in list(set(berndt_character_code.DISC))]))

In [169]:
clx_characters_used_in_data = set(''.join(list(clx_used_in_data)))
clx_characters_for_berndts_encoding = set(''.join([str(i) for i in list(set(berndt_character_code.CELEX))]))

In [141]:
disc_characters_used_in_data - disc_characters_for_berndts_encoding

{'#', '$', '7', '8'}

In [170]:
clx_characters_used_in_data - clx_characters_for_berndts_encoding

{'A'}

In [174]:
for i,word in enumerate(disc_encoded_words):
    for j in word:
        if any(x in j for x in ['#', '$', '7', '8']):
            print("Missing:", j, word, homophones_in_data_celex_mapped.word[pd.notnull(homophones_in_data_celex_mapped.disc)].iloc[i])

Missing: m$ ['m$', 'nIN'] morning
Missing: m$ ['m$', 'nIN'] morning
Missing: m$ ['m$', 'nIN'] morning
Missing: m$ ['m$', 'nIN'] morning
Missing: m$ ['m$', 'nIN'] morning
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: h7R ['h7R'] here
Missing: k$s ['k$s'] course
Missing: k$s ['k$s'] course
Missing: k$s ['k$s'] course
Missing: h7R ['h7R'] hear
Missing: h7R ['h7R'] hear
Missing: h7R ['h7R'] hear
Missing: h7R ['h7R'] hear
Missing: b8R ['b8R'] bear
Missing: b$d ['b$d'] board
Missing: b$d ['b$d'] board
Missing: b$d ['b$d'] board
Missing: p#st ['p#st'] past
Missing: p#st ['p#st'] past
Missing: p#st ['p#st'] past
Missing: p#st ['p#st'] past
Missing

In [173]:
for i,word in enumerate(clx_encoded_words):
    for j in word:
        if any(x in j for x in ['A']):
            print("Missing:", j, word, homophones_in_data_celex_mapped.word[pd.notnull(homophones_in_data_celex_mapped.clx)].iloc[i])

Missing: pA:st ['pA:st', ''] past
Missing: pA:st ['pA:st', ''] past
Missing: pA:st ['pA:st', ''] past
Missing: pA:st ['pA:st', ''] past
Missing: pA:st ['pA:st', ''] passed
Missing: pA:st ['pA:st', ''] passed


In [53]:
for i,word in enumerate(arpabet_encoded_words):
    for j in word:
        if j in ["EY2"]:
            print("Missing:", j, word, homophones_in_data.word.iloc[i])

Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1', 'N', 'S', 'EY2'] fiance
Missing: EY2 ['F', 'IY0', 'AA1'

In [11]:
"""ER1: heard, hurts
EH0: ensure
EY2: finance
AA2: lumbar
OW0 : cocoa
OW2: coco
"""

{'AA2', 'AO1', 'EH0', 'ER1', 'EY2', 'IY0', 'OW0', 'OW2'}

In [184]:
get_ARPAbet_phonetic_transcription(berndt_character_code.Example)

[['EY1', 'L'],
 ['AE1', 'D'],
 ['B', 'IY1'],
 ['EH1', 'N', 'D'],
 ['F', 'AA1', 'DH', 'ER0'],
 ['HH', 'AY1'],
 ['B', 'IH1', 'N'],
 ['B', 'OW1', 'T'],
 ['K', 'AA1', 'T'],
 ['S', 'AA1', 'F', 'T'],
 ['F', 'UW1', 'D'],
 ['HH', 'UH1', 'K'],
 ['Y', 'UW1', 'N', 'AY2', 'T'],
 ['AH1', 'P'],
 ['B', 'OY1'],
 ['AW1', 'T'],
 ['AH0', 'B', 'AW1', 'T'],
 ['B', 'AH1', 'T'],
 ['D', 'EY1'],
 ['F', 'AE1', 'N'],
 ['G', 'OW1'],
 ['HH', 'AE1', 'T'],
 ['JH', 'OW1', 'K'],
 ['K', 'IY1', 'P'],
 ['L', 'EY1', 'T'],
 ['M', 'AE1', 'N'],
 ['N', 'AA1', 'D'],
 ['P', 'EH1', 'N'],
 ['R', 'AE1', 'T'],
 ['S', 'UW1'],
 ['T', 'UW1'],
 ['V', 'AE1', 'N'],
 ['W', 'EY1', 'T'],
 ['Y', 'EH1', 'T'],
 ['Z', 'OW1', 'N'],
 ['CH', 'IH1', 'N'],
 ['AA1', 'K', 'S'],
 ['IH0', 'G', 'Z', 'IH1', 'S', 'T'],
 ['K', 'W', 'IH1', 'T'],
 ['P', 'AH1', 'D', 'AH0', 'L'],
 ['K', 'AE1', 'Z', 'AH0', 'M'],
 ['P', 'AA1', 'R', 'D', 'AH0', 'N'],
 ['S', 'IH1', 'NG'],
 ['SH', 'IY1'],
 ['TH', 'IH1', 'N'],
 ['DH', 'EH1', 'N'],
 ['R', 'UW1', 'ZH'],
 ['AA1', 'N', '

In [185]:
for i, word in enumerate(get_ARPAbet_phonetic_transcription(berndt_character_code.Example)):
    for j in word:
        if j in ['IH0']:
            print(word, berndt_character_code.Example[i])

['IH0', 'G', 'Z', 'IH1', 'S', 'T'] exist


In [190]:
print(get_ARPAbet_phonetic_transcription(["ringing"]))
print(get_ARPAbet_phonetic_transcription(["bin"]))


[['R', 'IH1', 'NG', 'IH0', 'NG']]
[['B', 'IH1', 'N']]


In [194]:
print(get_ARPAbet_phonetic_transcription(["bee"]))
print(get_ARPAbet_phonetic_transcription(["chilly"]))
print(get_ARPAbet_phonetic_transcription(["greedy"]))

[['B', 'IY1']]
[['CH', 'IH1', 'L', 'IY0']]
[['G', 'R', 'IY1', 'D', 'IY0']]


In [198]:
print(get_ARPAbet_phonetic_transcription(["soft"]))
print(get_ARPAbet_phonetic_transcription(["board"]))
print(get_ARPAbet_phonetic_transcription(["course"]))
print(get_ARPAbet_phonetic_transcription(["extraordinary"]))

[['S', 'AA1', 'F', 'T']]
[['B', 'AO1', 'R', 'D']]
[['K', 'AO1', 'R', 'S']]
[['IH0', 'K', 'S', 'T', 'R', 'AO1', 'R', 'D', 'AH0', 'N', 'EH2', 'R', 'IY0']]


In [36]:
print(get_ARPAbet_phonetic_transcription(["ensure"]))
print(get_ARPAbet_phonetic_transcription(["endurance"]))
print(get_ARPAbet_phonetic_transcription(["engage"]))
print(get_ARPAbet_phonetic_transcription(["enable"]))
print(get_ARPAbet_phonetic_transcription(["end"]))
print(get_ARPAbet_phonetic_transcription(["edge"]))
print(get_ARPAbet_phonetic_transcription(["enter"]))
print(get_ARPAbet_phonetic_transcription(["enrooted"]))
print(get_ARPAbet_phonetic_transcription(["encore"]))

[['EH0', 'N', 'SH', 'UH1', 'R']]
[['EH1', 'N', 'D', 'ER0', 'AH0', 'N', 'S']]
[['EH0', 'N', 'G', 'EY1', 'JH']]
[['EH0', 'N', 'EY1', 'B', 'AH0', 'L']]
[['EH1', 'N', 'D']]
[['EH1', 'JH']]
[['EH1', 'N', 'T', 'ER0']]
[['EH0', 'N', 'R', 'UW1', 'T', 'UW2', 'D']]
[['AA1', 'N', 'K', 'AO2', 'R']]


In [50]:
print(get_ARPAbet_phonetic_transcription(["earn"]))
print(get_ARPAbet_phonetic_transcription(["father"]))

[['ER1', 'N']]
[['F', 'AA1', 'DH', 'ER0']]


In [77]:
print(get_ARPAbet_phonetic_transcription(["fiance"]))
print(get_ARPAbet_phonetic_transcription(["educate"]))
print(get_ARPAbet_phonetic_transcription(["exaggerate"]))
print(get_ARPAbet_phonetic_transcription(["survey"]))
print(get_ARPAbet_phonetic_transcription(["bay"]))
print(get_ARPAbet_phonetic_transcription(["convey"]))
print(get_ARPAbet_phonetic_transcription(["prey"]))
print(get_ARPAbet_phonetic_transcription(["gray"]))
print(get_ARPAbet_phonetic_transcription(["purvey"]))
print(get_ARPAbet_phonetic_transcription(["thursday"]))
print(get_ARPAbet_phonetic_transcription(["day"]))

[['F', 'IY0', 'AA1', 'N', 'S', 'EY2']]
[['EH1', 'JH', 'AH0', 'K', 'EY2', 'T']]
[['IH0', 'G', 'Z', 'AE1', 'JH', 'ER0', 'EY2', 'T']]
[['S', 'ER1', 'V', 'EY2']]
[['B', 'EY1']]
[['K', 'AH0', 'N', 'V', 'EY1']]
[['P', 'R', 'EY1']]
[['G', 'R', 'EY1']]
[['P', 'ER0', 'V', 'EY1']]
[['TH', 'ER1', 'Z', 'D', 'EY2']]
[['D', 'EY1']]


In [ ]:
for i, word in enumerate(get_ARPAbet_phonetic_transcription(berndt_character_code.Example)):
    for j in word:
        if j in ['IH0']:
            print(word, berndt_character_code.Example[i])

In [40]:
berndt_conditional_probs_words = get_ARPAbet_phonetic_transcription(berndt_conditional_probs.Example)
arpabet_used_in_bernd_examples = set(sum(berndt_conditional_probs_words,[]))

In [49]:
for i,word in enumerate(berndt_conditional_probs_words):
    for j in word:
        if j in ["ER1", "EY2"]:
            print(i,j, word, berndt_conditional_probs.Example.iloc[i])

58 ER1 ['ER1', 'N'] earn
63 ER1 ['HH', 'ER1', 'S'] hearse
103 ER1 ['AH0', 'F', 'ER1', 'M'] affirm 
110 ER1 ['D', 'ER1', 'JH'] dirge
140 ER1 ['W', 'ER1', 'S'] worse
166 ER1 ['S', 'K', 'ER1', 'JH'] scourge
182 ER1 ['S', 'ER1', 'V', 'EY2'] survey
182 EY2 ['S', 'ER1', 'V', 'EY2'] survey
234 EY2 ['EH1', 'JH', 'AH0', 'K', 'EY2', 'T'] educate
251 EY2 ['IH0', 'G', 'Z', 'AE1', 'JH', 'ER0', 'EY2', 'T'] exaggerate
298 ER1 ['P', 'ER1'] purr 
305 ER1 ['D', 'IH0', 'S', 'ER1', 'N'] discern
315 ER1 ['D', 'IH0', 'Z', 'ER1', 'T'] dessert


In [42]:
arpabet_used_in_data - arpabet_used_in_bernd_examples

{'AA2', 'OW2'}